In [881]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [883]:
pip install elasticsearch

Note: you may need to restart the kernel to use updated packages.


In [885]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [889]:
from openai import OpenAI
client = OpenAI()

In [1029]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

In [1031]:
es_client.info()

ObjectApiResponse({'name': '64c3ceb251b3', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'rUhQ_EYDQYyrOqzyZprqaw', 'version': {'number': '8.17.6', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7', 'build_date': '2025-04-30T14:07:12.231372970Z', 'build_snapshot': False, 'lucene_version': '9.12.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [1033]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "question": {"type": "text"}
        }
    }
}

In [1035]:
index_name = "course-questions"

In [1038]:
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [1041]:
from tqdm.auto import tqdm

In [1043]:
# Q2. Indexing the data. Answer: index
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [1045]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "filter": {
                    "term": {
                        "course.keyword": "machine-learning-zoomcamp",
                    }
                },
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    return response
    # result_docs = []

    # for hit in response['hits']['hits']:
    #     result_docs.append(hit['_source'])
    # return [response, result_docs]

In [1051]:
# Q3. Searching. Answer: 44.50
query1 = "How do execute a command on a Kubernetes pod?"
response = elastic_search(query1)
response['hits']['hits'][0]['_score']

44.50556

In [1059]:
result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [1061]:
# Q4. Filtering. Answer: 'How do I copy files from a different folder into docker container’s working directory?'
query = "How do copy a file to a Docker container?"
result_docs[2]['question']

'How do I copy files from a different folder into docker container’s working directory?'

In [1063]:
def build_propmt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
    """.strip()
    
    context = ""
    
    for doc in search_results:
        context = context + f"Q: {doc['question']}\nA: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [1067]:
# Q5. Building a prompt. Answer: 1446
prompt = build_propmt(query, result_docs)
print(len(prompt))

1446


In [1069]:
pip install tiktoken

Note: you may need to restart the kernel to use updated packages.


In [1091]:
import tiktoken

In [1093]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [1095]:
token_ids = encoding.encode(prompt)

In [1097]:
# Q6. Tokens. Answer: 320
num_tokens = len(token_ids)
num_tokens

320

In [1099]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{'role': 'user', 'content': prompt}]
    )
    return response.choices[0].message.content

In [1101]:
# Bonus: generating the answer (ungraded)
answer = llm(prompt)
print(answer)

You can copy files from your local machine into a Docker container using the `docker cp` command. The basic syntax is:

```
docker cp /path/to/local/file_or_directory container_id:/path/in/container
```

This command allows you to copy a file or directory from your local machine into a running Docker container.


In [1109]:
# Bonus: calculating the costs (ungraded)
in_token_price = 0.005 / 1000
out_token_price = 0.015 / 1000

out_token_ids = encoding.encode(answer)
num_out_tokens = len(out_token_ids)

total_cost_in = num_tokens * in_token_price
total_cost_out = num_out_tokens * out_token_price
print(f"Input = {total_cost_in}, Output = {total_cost_out}, \nTotal = {total_cost_in + total_cost_out}")

Input = 0.0016, Output = 0.000975, 
Total = 0.002575
